## Part 1: Preprocessing

In [1]:
# Import our dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')

# Display the first five rows
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Select two y target columns
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

# Display the first five rows
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
columns_for_X = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
                 'OverTime', 'StockOptionLevel', 'WorkLifeBalance',
                 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[columns_for_X]

# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [5]:
# Check the unique values in the OverTime column of X_df
X_df['OverTime'].unique()

array(['Yes', 'No'], dtype=object)

In [6]:
# Encode categorical feaature 'OverTime' in X (before splitting)

# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

ohe_overtime = OneHotEncoder(drop='if_binary', sparse_output=False)
overtime_encoded = ohe_overtime.fit_transform(X_df[['OverTime']])

# Drop original categorical column and add the new one-hot encoded column
X_df = X_df.drop(columns=['OverTime'])
X_df = pd.concat([X_df, pd.DataFrame(overtime_encoded,\
                                      columns=["OverTime_yes"])], axis=1)

In [7]:
# Check the unique values after encoding
X_df['OverTime_yes'].value_counts()

OverTime_yes
0.0    1054
1.0     416
Name: count, dtype: int64

In [8]:
# Show the data types for X_df to ensure all integers
X_df.dtypes

Education                    int64
Age                          int64
DistanceFromHome             int64
JobSatisfaction              int64
StockOptionLevel             int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
NumCompaniesWorked           int64
OverTime_yes               float64
dtype: object

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)

In [10]:
# Check the unique values in the Department column of y_df
y_df['Department'].unique()

array(['Sales', 'Research & Development', 'Human Resources'], dtype=object)

In [11]:
# Check the count
y_df['Department'].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [12]:
# Check the unique values in the Attrition column of y_df
y_df['Attrition'].unique()

array(['Yes', 'No'], dtype=object)

In [13]:
# Check the count
y_df['Attrition'].value_counts()

Attrition
No     1233
Yes     237
Name: count, dtype: int64

In [14]:
# Encode two y target variables (after splitting)
# First, preprocess Department

# Create a OneHotEncoder for the Department column
ohe_dept = OneHotEncoder(sparse_output=False)

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept = ohe_dept.fit_transform(y_train[['Department']])
y_test_dept = ohe_dept.transform(y_test[['Department']])

y_train_dept

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [15]:
# Create a OneHotEncoder for the Attrition column

# Fit the encoder to the training data
ohe_attr = OneHotEncoder(sparse_output=False)

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr = ohe_attr.fit_transform(y_train[['Attrition']])
y_test_attr = ohe_attr.transform(y_test[['Attrition']])

y_train_attr

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [16]:
# Standarize X Features (after splitting)

# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Part 2: Create, Compile, and Train the Model

In [17]:
# Find the number of columns in the X training data.
input_shape = X_train_scaled.shape[1]
print(input_shape)

# Create the input layer
input_layer = Input(shape=(input_shape,), name='input')

# Create at least two shared layers
shared_layer1 = Dense(128, activation='relu', name='shared1')(input_layer)
shared_layer2 = Dense(64, activation='relu', name='shared2')(shared_layer1)

10


In [18]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = Dense(32, activation='relu', name='department_hidden')(shared_layer2)

# Create the output layer
dept_output = Dense(y_train_dept.shape[1], activation='softmax',\
                      name='department_output')(dept_hidden)

In [19]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_hidden = Dense(32, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attr_output = Dense(y_train_attr.shape[1], activation='sigmoid',\
                      name='attrition_output')(attr_hidden)

In [20]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attr_output])

# Compile the model
model.compile(optimizer='adam',
              loss=['categorical_crossentropy', 'binary_crossentropy'],
              metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 10)]         0           []                               
                                                                                                  
 shared1 (Dense)                (None, 128)          1408        ['input[0][0]']                  
                                                                                                  
 shared2 (Dense)                (None, 64)           8256        ['shared1[0][0]']                
                                                                                                  
 department_hidden (Dense)      (None, 32)           2080        ['shared2[0][0]']                
                                                                                              

In [21]:
# Train the model
history = model.fit(X_train_scaled, [y_train_dept, y_train_attr],
                    epochs=50, batch_size=16, verbose=1)

Epoch 1/50
69/69 [==============================] - 1s 2ms/step - loss: 1.3870 - department_output_loss: 0.8638 - attrition_output_loss: 0.5231 - department_output_accuracy: 0.6107 - attrition_output_accuracy: 0.8113
Epoch 2/50
69/69 [==============================] - 0s 2ms/step - loss: 1.1893 - department_output_loss: 0.7716 - attrition_output_loss: 0.4176 - department_output_accuracy: 0.6543 - attrition_output_accuracy: 0.8294
Epoch 3/50
69/69 [==============================] - 0s 2ms/step - loss: 1.1460 - department_output_loss: 0.7642 - attrition_output_loss: 0.3819 - department_output_accuracy: 0.6543 - attrition_output_accuracy: 0.8430
Epoch 4/50
69/69 [==============================] - 0s 2ms/step - loss: 1.1290 - department_output_loss: 0.7589 - attrition_output_loss: 0.3701 - department_output_accuracy: 0.6534 - attrition_output_accuracy: 0.8612
Epoch 5/50
69/69 [==============================] - 0s 2ms/step - loss: 1.0998 - department_output_loss: 0.7478 - attrition_output_l

In [22]:
# Evaluate the model with the testing data
eval_results = model.evaluate(X_test_scaled, [y_test_dept, y_test_attr])

12/12 [==============================] - 0s 2ms/step - loss: 2.3214 - department_output_loss: 1.4751 - attrition_output_loss: 0.8463 - department_output_accuracy: 0.5924 - attrition_output_accuracy: 0.8370


In [23]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {eval_results[3]:.4f}")
print(f"Attrition Accuracy: {eval_results[4]:.4f}") 

Department Accuracy: 0.5924
Attrition Accuracy: 0.8370


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. While accuracy is a commonly used metric, it should not be used alone.  For the Attrition prediction (binary classification), accuracy could be misleading if the classes are imbalanced.  About 16% of the employees experience attrition (y=1).  Metrics like precision, recall, or F1-score would be useful, as they better capture the model's ability to correctly predict the minority class.  
2. For the Department output layer, softmax activation was chosen becuase it is a multi-class classification problem (R&D, Sales and HR).  The softmax function ouputs the probabilities for each of the three possible departments.  Thus, the department with the highest probability is given.

    For the Attrition output layer, sigmoid activation was chosen because it is a binary classification problem.  Either the employee leaves (y=1) or stays (y=0).  The sigmoid function outputs a probability between 0 and 1, ideal for predicting binary outcomes.

    The combination of softmax for Department and sigmoid for Attrition ensures that both types of classification are handled correctly.

3. Here are three ways the model can be improved.  (1) Class imalance handling for the X dataset.  (2)  Huyperparameter tuning.  The model's performance can be improved by tuning the number of layers, numer of neurons per layer, and batch sie.  (3)  Additional features or feature engineering.  More meaningful features would help.  Creating new columns from existing columns may help based on domain knowledge.